- evaluate GRN

- remake Fig 4 

In [1]:
import os
import sys
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import celloracle as co

In [2]:
plt.rcParams["font.family"] = "arial"
plt.rcParams["figure.figsize"] = [9,6]
%config InlineBackend.figure_format = 'retina'
plt.rcParams["savefig.dpi"] = 600
%matplotlib inline

In [3]:
# Make folder to save plots
save_folder = "../results"

In [4]:
intermediate_dir = "/Users/LoboM/Dropbox/FINAL_CO/"

In [5]:
co_links = co.load_hdf5(file_path=intermediate_dir + "/links.celloracle.links")

- use output of Seurat FindAllMarkers(atlas_rna, logfc.threshold = 0.25,return.thresh = 0.01)
to get top markers

In [75]:
top_markers = pd.read_csv("/Users/LoboM/Documents/ArchR_analysis/notebooks/top_markers_all_clusters.csv")

In [76]:
# correct hyphen issue
for idx in top_markers['gene'].index:
    entry = top_markers.loc[idx,'gene']
    if '.' in entry:
        if 'Rik' not in entry and 'Rp' not in entry:
            # print(entry)
            entry1 = entry.replace('.','-')  
            print(entry,entry1)
            top_markers.loc[idx,'gene'] = entry1

Nkx2.3 Nkx2-3
Nkx2.6 Nkx2-6
Nkx2.6 Nkx2-6
mt.Co2 mt-Co2
mt.Atp6 mt-Atp6
mt.Cytb mt-Cytb
mt.Nd4 mt-Nd4
mt.Co3 mt-Co3
mt.Co1 mt-Co1
mt.Nd3 mt-Nd3
mt.Nd1 mt-Nd1
mt.Nd2 mt-Nd2
Nkx2.6 Nkx2-6
mt.Co1 mt-Co1
mt.Co3 mt-Co3
mt.Co2 mt-Co2
Nkx2.5 Nkx2-5
Nkx2.3 Nkx2-3
H2.Ab1 H2-Ab1
mt.Co1 mt-Co1
mt.Co2 mt-Co2
H2.Eb1 H2-Eb1
Nkx6.2 Nkx6-2
mt.Co3 mt-Co3
Nkx2.6 Nkx2-6
Nkx2.3 Nkx2-3
H2.Q2 H2-Q2
Nkx2.6 Nkx2-6
mt.Cytb mt-Cytb
Nkx2.3 Nkx2-3
mt.Nd3 mt-Nd3
mt.Co3 mt-Co3
mt.Co2 mt-Co2
mt.Cytb mt-Cytb
mt.Nd3 mt-Nd3
mt.Co1 mt-Co1
Nkx2.6 Nkx2-6
mt.Nd4 mt-Nd4
mt.Nd2 mt-Nd2
Nkx2.3 Nkx2-3
mt.Nd5 mt-Nd5
Nkx2.6 Nkx2-6
Nkx2.6 Nkx2-6
Nkx2.3 Nkx2-3
mt.Atp6 mt-Atp6
Nkx2.3 Nkx2-3
Nkx2.3 Nkx2-3
mt.Co1 mt-Co1
Nkx2.6 Nkx2-6
mt.Atp6 mt-Atp6
mt.Nd4 mt-Nd4
mt.Cytb mt-Cytb
Nkx2.6 Nkx2-6
Nkx2.1 Nkx2-1
Nkx2.3 Nkx2-3
Nkx2.5 Nkx2-5
Nkx2.5 Nkx2-5
mt.Co1 mt-Co1
Nkx2.6 Nkx2-6
mt.Atp6 mt-Atp6
mt.Co3 mt-Co3
mt.Nd3 mt-Nd3
mt.Nd1 mt-Nd1
mt.Cytb mt-Cytb
mt.Nd4 mt-Nd4
mt.Co2 mt-Co2
mt.Nd2 mt-Nd2
mt.Co1 mt-Co1
mt.Co3 mt-Co3
mt.Nd3 mt-Nd3
mt.N

In [77]:
unfiltered = co_links.links_dict.copy()

In [79]:
top_markers_pos = top_markers[top_markers['avg_logFC'] > 0] # remove downreg genes

In [82]:
# filter edges when both source and target are present in up/down genes and -log10p-val of edge > 2
logp_cutoff = 3
edge_cutoff = 0.005
for clus_int in range(28):
    clus = str(clus_int)
    print(clus)
    #print(unfiltered[clus].shape)
    top_markers_clus = list(top_markers_pos.loc[top_markers_pos['cluster']==clus_int,'gene'])
    tmp_grn = unfiltered[clus].copy()
    tmp_grn = tmp_grn[((tmp_grn['source'].isin(top_markers_clus)) & (tmp_grn['-logp'] > logp_cutoff) & (tmp_grn['coef_abs'] > edge_cutoff))]
    
    unfiltered[clus] = tmp_grn.copy()
    print(unfiltered[clus].shape)

0
(16252, 6)
1
(5686, 6)
2
(4899, 6)
3
(6561, 6)
4
(15273, 6)
5
(18229, 6)
6
(20310, 6)
7
(9000, 6)
8
(8780, 6)
9
(16559, 6)
10
(3799, 6)
11
(18027, 6)
12
(6673, 6)
13
(6665, 6)
14
(3221, 6)
15
(14616, 6)
16
(4636, 6)
17
(12284, 6)
18
(19867, 6)
19
(10994, 6)
20
(13166, 6)
21
(10704, 6)
22
(22597, 6)
23
(5536, 6)
24
(9236, 6)
25
(20223, 6)
26
(9774, 6)
27
(7996, 6)


In [83]:
# update links object
for clus_int in range(28):
    clus = str(clus_int)
    print(co_links.filtered_links[clus].shape)
    co_links.filtered_links[clus] = unfiltered[clus].copy()
    print(co_links.filtered_links[clus].shape)

(30000, 6)
(16252, 6)
(30000, 6)
(5686, 6)
(30000, 6)
(4899, 6)
(30000, 6)
(6561, 6)
(30000, 6)
(15273, 6)
(30000, 6)
(18229, 6)
(30000, 6)
(20310, 6)
(30000, 6)
(9000, 6)
(30000, 6)
(8780, 6)
(30000, 6)
(16559, 6)
(30000, 6)
(3799, 6)
(30000, 6)
(18027, 6)
(30000, 6)
(6673, 6)
(30000, 6)
(6665, 6)
(30000, 6)
(3221, 6)
(30000, 6)
(14616, 6)
(30000, 6)
(4636, 6)
(30000, 6)
(12284, 6)
(30000, 6)
(19867, 6)
(30000, 6)
(10994, 6)
(30000, 6)
(13166, 6)
(30000, 6)
(10704, 6)
(30000, 6)
(22597, 6)
(30000, 6)
(5536, 6)
(30000, 6)
(9236, 6)
(30000, 6)
(20223, 6)
(30000, 6)
(9774, 6)
(30000, 6)
(7996, 6)


In [84]:
co_links.get_score()

processing... batch 1/2
0: finished.
1: finished.
10: finished.
11: finished.
12: finished.
13: finished.
14: finished.
15: finished.
16: finished.
17: finished.
18: finished.
19: finished.
2: finished.
20: finished.
21: finished.
22: finished.
processing... batch 2/2
23: finished.
24: finished.
25: finished.
26: finished.
27: finished.
3: finished.
4: finished.
5: finished.
6: finished.
7: finished.
8: finished.
9: finished.


In [85]:
co_links.to_hdf5(file_path=intermediate_dir + "/links_filtered_top_in_source_0005.celloracle.links")
oracle = co.load_hdf5(intermediate_dir + "/oracle_initial.celloracle.oracle")
# co_links = co.load_hdf5(file_path=save_folder + "/links_10k.celloracle.links")


In [86]:
oracle.get_cluster_specific_TFdict_from_Links(links_object=co_links)
oracle.fit_GRN_for_simulation(alpha=10, use_cluster_specific_TFdict=True)
oracle.to_hdf5(intermediate_dir + "oracle_filtered_top_in_source_0005.celloracle.oracle")                                                  

fitting GRN again...
calculating GRN in 0



genes_in_gem: 10000
models made for 5369 genes
calculating GRN in 1



genes_in_gem: 10000
models made for 3623 genes
calculating GRN in 10



genes_in_gem: 10000
models made for 2963 genes
calculating GRN in 11



genes_in_gem: 10000
models made for 5533 genes
calculating GRN in 12


...


genes_in_gem: 10000
models made for 5626 genes
calculating GRN in 6



genes_in_gem: 10000
models made for 5388 genes
calculating GRN in 7



genes_in_gem: 10000
models made for 4334 genes
calculating GRN in 8



genes_in_gem: 10000
models made for 4569 genes
calculating GRN in 9



genes_in_gem: 10000
models made for 5456 genes


In [87]:
goi = "Foxn1"
# Enter perturbation conditions to simulate signal propagation after the perturbation.
oracle.simulate_shift(perturb_condition={goi: 0.0},
                      n_propagation=5) # tested separately

foxn1_ko_sim = pd.DataFrame(data=oracle.adata.layers['simulated_count'], 
             index=oracle.adata.obs_names,
             columns=oracle.adata.var_names)

foxn1_ko_sim.to_csv(os.path.join(intermediate_dir, 'Foxn1_ko_10k_5_filtered_top_in_source_0005.csv'))

oracle.adata.write_h5ad(intermediate_dir + "/co_foxn1_ko_sim_10k_5_prop_filtered_top_in_source_0005.h5ad")



NameError: name 'adata' is not defined

In [88]:
ctrl_sim = pd.DataFrame(oracle.adata.layers['simulation_input'], 
             index=oracle.adata.obs_names,
            columns=oracle.adata.var_names)

ctrl_sim.to_csv(os.path.join(intermediate_dir, goi + '_no_perturbation_10k_5_filtered_top_in_source_0005.csv'))






In [89]:
fox_n1_ko_act = pd.read_csv("/Users/LoboM//Dropbox/scRNA_data_analysis/pharynx_analysis3/results/preprocess_explore_10x/foxn1_ko/2020_Apr_13__17_30_19__foxn1_ko_isolate_via_classifier/user/foxn1_ko_diff_expr.csv",index_col=0)
import scanpy as sc
adata3 = sc.read_h5ad("/Users/LoboM/Dropbox/pharynx_celloracle/objects/roadmap/RoadmapSeurat3.rds.h5ad")
genes_to_use = list(adata3[:,adata3.var['variable_gene']==True].var.index)
genes_to_use_int = list(set(genes_to_use).intersection(set(fox_n1_ko_act.index)))
cells_to_use = list(adata3[(adata3.obs['seurat_clusters'] == '4') | (adata3.obs['seurat_clusters'] == '9') | (adata3.obs['seurat_clusters'] == '25')].obs.index)
#foxn1_file = intermediate_dir + '/' + "Foxn1_ko_10k_5_FINAL.csv"
#import pandas as pd
#foxn1_ko_sim = pd.read_csv(foxn1_file,index_col=0)
#ctrl_file = intermediate_dir + '/' + "no_perturbation_10k_5_FINAL.csv"
#ctrl_sim = pd.read_csv(ctrl_file,index_col=0)
genes_to_use_int = list(set(genes_to_use_int).intersection(set(foxn1_ko_sim.columns)))
foxn1_ko_sim_trunc = foxn1_ko_sim.loc[cells_to_use,genes_to_use_int]
ctrl_sim_trunc = ctrl_sim.loc[cells_to_use,genes_to_use_int]
diff_ko_ctrl = foxn1_ko_sim_trunc - ctrl_sim_trunc
diff_ko_ctrl_mean = diff_ko_ctrl.mean()
fox_n1_ko_act_trunc = fox_n1_ko_act.loc[genes_to_use_int,'logFC']
import scipy
rho, pval = scipy.stats.spearmanr(list(diff_ko_ctrl_mean.values),list(fox_n1_ko_act_trunc.values))

In [90]:
rho

0.3238076334384306

In [101]:
co_links.merged_score.to_csv(intermediate_dir +"/Table_celloracle.csv")

In [116]:
delta_sim = foxn1_ko_sim - ctrl_sim

In [117]:
delta_sim.to_csv(os.path.join(intermediate_dir, goi + '_delta_sim_filtered_top_in_source_0005.csv'))




In [ ]:
os.makedirs(f"{intermediate_dir}/filtered_GRN_10k_0701_2021", exist_ok = True)
for cluster in co_links.links_dict.keys():
    co_links.filtered_links[cluster].to_csv(f"{intermediate_dir}/filtered_GRN_10k_0701_2021/{cluster}.csv")